In [5]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import fastparquet
import sys

In [1]:
class exc(Exception):
    pass

In [2]:
def get_html(url,headers):
    num_retries=0
    max_retries=3
    try:
        while num_retries <= max_retries:
            r = requests.get(url,headers=headers)
            if r.status_code == requests.codes.ok:
                return r
            else:
                time.sleep(3)
                if num_retries == max_retries:
                    raise exc
                num_retries += 1
    except exc:
        print('Http_error'+url)
        raise
    except:
        print('Unexcepted error:', sys.exc_info()[0])
        raise

In [3]:
def parse_html(r):
    soup= Beautifulsoup(r.text,'lxml')
    if soup != None :
        return soup
    else:
        raise exc
    

In [6]:
url = 'https://www.sum.com.tw/carsearchlist.php?v=3&ctp_prt=&price1=0&price2=220&yearrange=&cararea=&carregion=&cartype=&carcolor=&brand=&model=&year1=&year2=&doors=&q=&show_type=&carrecommend=&comp_goldstore=&discount=&cc1=&cc2=&protection=&carvr=&page=&sort=&sortpage1=&gclid=&priceRange1=&priceRange2=&wd=&_de=0.5690229095899482&page=1'
# https://www.sum.com.tw/carsearchlist.php?v=3&ctp_prt=&price1=0&price2=220&yearrange=&cararea=&carregion=&cartype=&carcolor=&brand=&model=&year1=&year2=&doors=&q=&show_type=&carrecommend=&comp_goldstore=&discount=&cc1=&cc2=&protection=&carvr=&page=&sort=&sortpage1=&gclid=&priceRange1=&priceRange2=&wd=&_de=0.5690229095899482&page=2b
headers={
    'Host':'www.sum.com.tw',
'Connection':'keep-alive',
'Accept':'text/plain, */*; q=0.01',
'X-Requested-With':'XMLHttpRequest',
'sec-ch-ua-mobile':'?0',
'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36',
# 'Sec-Fetch-Site':'same-origin',
# 'Sec-Fetch-Mode':'cors',
# 'Sec-Fetch-Dest':'empty',
# 'Referer':'https://www.sum.com.tw/carsearch.php?v=3&ctp_prt=&price1=0&price2=220&yearrange=&cararea=&carregion=&cartype=&carcolor=&brand=&model=&year1=&year2=&doors=&q=&show_type=&carrecommend=&comp_goldstore=&discount=&cc1=&cc2=&protection=&carvr=&page=&sort=&sortpage1=&gclid=&priceRange1=&priceRange2=&wd=&_de=0.5690229095899482&page=1',
'Accept-Encoding':'gzip, deflate, br',
'Accept-Language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
'Cookie':'PHPSESSID=qn203skntjuv8bkevorgrhg5g7; _gcl_au=1.1.1197458183.1624031200; _ga=GA1.3.849228594.1624031200; __auc=38660c2a17a1fcd839cb333ad40; _hjTLDTest=1; _hjid=13961593-1974-490c-8e9b-f0726e39cde0; _fbp=fb.2.1624031200855.1701953032; _gid=GA1.3.966926215.1624256678; _TUCI_T=sessionNumber+18799&pageView+18799; Tagtoo_pta=pta_03+_&lta+_tm^3Ad1056_; __asc=c83520af17a2d7bcacaaa8c8958; _TUCS=1; _TUCI=sessionNumber+2498&ECId+155&hostname+www.sum.com.tw&pageView+7493',
}
r=requests.get(url,headers=headers)

In [18]:
soup=BeautifulSoup(r.text,'lxml')
seo_list=soup.find_all('li',class_='seo_list')
list1=[]
df1=pd.DataFrame(columns=['id','car_brand','car_model','car_address','car_title','dealer_name'])

for li in seo_list:
    idnum=li.get('data-seo_name')
    car_info=li.find('h3').a.text.split('｜')

    
    car_brand=car_info.split(' ',1)[0]
    car_model=car_info.split(' ',1)[1]
    car_address=li.find('ul').find_next_sibling().find('li').text
    car_title=li.find('div',class_='carCondition').h3.text.strip()
    car_dealer=li.find('ul').find_next_sibling().find('li').find_next_sibling().text
    df1= df1.append({'id':idnum,'car_brand':car_brand,'car_model':car_model,
                     'car_address':car_address,'car_title':car_title,'dealer_name':car_dealer}, ignore_index=True)

# df1.to_parquet('sum_car.parquet',engine='fastparquet')
df1

['TOYOTA VIOS ', '2016']
['MITSUBISHI OUTLANDER ', '2017']
['LUXGEN S5 TURBO ECO HYPER', '2016']
['HONDA CR-V ', '2017']
['HONDA HR-V', '2018']
['SUBARU FORESTER ', '2018']
['TOYOTA SIENTA', '2019']
['FORD ECOSPORT ', '2017']
['TOYOTA CAMRY ', '2019']
['NISSAN TIIDA 5D ', '2012']
['BENZ A-CLASS W176', '2014']
['TOYOTA WISH ', '2016']
['TOYOTA RAV4 ', '2019']
['MITSUBISHI ZINGER', '2018']
['HONDA CIVIC', '2011']
['TOYOTA ALTIS ', '2018']
['TOYOTA RAV4 ', '2018']
['HONDA CR-V ', '2017']
['LUXGEN U6 GT', '2020']
['BENZ CLA-CLASS', '2017']
['NISSAN SUPER SENTRA ', '2016']
['TOYOTA C-HR', '2019']
['TOYOTA ALTIS ', '2017']
['LUXGEN U7 TURBO ', '2014']
['TOYOTA ALTIS ', '2013']
['TOYOTA VIOS ', '2010']
['TOYOTA ALTIS ', '2017']
['HYUNDAI SUPER ELANTRA ', '2018']
['MITSUBISHI COLT PLUS ', '2018']
['BENZ GLA-CLASS X156 ', '2015']


,id,car_brand,car_model,car_address,car_title,dealer_name
0,575804,TOYOTA,VIOS ｜2016,臺南市,可網購!只跑1.7萬 17年式改版 手自排 新引擎 ABS 雙安 代步好選擇 可全貸 不用殺...,金台南汽車(大同一店)
1,575662,MITSUBISHI,OUTLANDER ｜2017,臺中市,【全台最便宜不用再看了】僅跑5萬原版件認證車 正17年 四輪傳動 前為公家機關主管座車 大螢...,永貿汽車
2,575654,LUXGEN,S5 TURBO ECO HYPER｜2016,臺中市,僅跑4.1萬公里，小改款，新車頭，ECO版，更省油、實惠，一手車，低里程，渦輪增壓，定速、循跡防滑,億昇汽車
3,575640,HONDA,CR-V ｜2017,臺南市,可網購!18年式 S版 大包 ACC 電動尾門 電動椅 全景天窗 車道偏移 少跑里程保證 全...,金台南汽車(大同一店)
4,575624,HONDA,HR-V｜2018,臺南市,可網購!閃蝶藍 S旗艦版 一手車 原廠影音 LED 八安 全車原鈑件 只跑長途 可全貸 不...,金台南汽車(大同一店)
5,575617,SUBARU,FORESTER ｜2018,臺南市,特價!AWD 19年式新款 最頂級 i-S EYESIGHT 只跑2.8萬里程保證 ACC ...,金台南汽車(大同一店)
6,575573,TOYOTA,SIENTA｜2019,臺中市,【特價–不用殺價就超便宜】正2019年 僅跑1萬多! 認證車 1800CC有力省稅 原廠新車...,永貿汽車
7,575505,FORD,ECOSPORT ｜2017,臺南市,可網購!原裝進口 一手只跑1.5萬 根本新車 TCS 六安 原廠保養 全車原鈑件 可全貸 不...,金台南汽車(大同一店)
8,575486,TOYOTA,CAMRY ｜2019,臺中市,【全台最便宜不用再看了】僅跑2萬 原廠新車保固至2023/05 原版原漆已認證 真的跟新車沒...,永貿汽車
9,575472,NISSAN,TIIDA 5D ｜2012,高雄市,正一手用車 小車大空間 恆溫 抬頭顯示 ABS 僅跑8萬多公里 不用殺價就很便宜,君豪汽車


In [12]:
np = df1['id'][0]
url_in = 'https://www.sum.com.tw/carinfo-573699.php'
headers={
    'Host':'www.sum.com.tw',
'Connection':'keep-alive',
'Accept':'text/plain, */*; q=0.01',
# 'X-Requested-With':'XMLHttpRequest',
# 'sec-ch-ua-mobile':'?0',
'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36',
# 'Sec-Fetch-Site':'same-origin',
# 'Sec-Fetch-Mode':'cors',
# 'Sec-Fetch-Dest':'empty',
# 'Referer':'https://www.sum.com.tw/carsearch.php?v=3&ctp_prt=&price1=0&price2=220&yearrange=&cararea=&carregion=&cartype=&carcolor=&brand=&model=&year1=&year2=&doors=&q=&show_type=&carrecommend=&comp_goldstore=&discount=&cc1=&cc2=&protection=&carvr=&page=&sort=&sortpage1=&gclid=&priceRange1=&priceRange2=&wd=&_de=0.5690229095899482&page=1',
# 'Accept-Encoding':'gzip, deflate, br',
# 'Accept-Language':'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
'Cookie':'PHPSESSID=qn203skntjuv8bkevorgrhg5g7; _gcl_au=1.1.1197458183.1624031200; _ga=GA1.3.849228594.1624031200; __auc=38660c2a17a1fcd839cb333ad40; _hjTLDTest=1; _hjid=13961593-1974-490c-8e9b-f0726e39cde0; _fbp=fb.2.1624031200855.1701953032; _gid=GA1.3.966926215.1624256678; _TUCI_T=sessionNumber+18799&pageView+18799; Tagtoo_pta=pta_03+_&lta+_tm^3Ad1056_; __asc=c83520af17a2d7bcacaaa8c8958; _TUCS=1; _TUCI=sessionNumber+2498&ECId+155&hostname+www.sum.com.tw&pageView+7493',
}
# id','m_id','auto_brand_en','auto_brand_id','auto_gas_size','auto_make_date','auto_mileage_num','auto_price','auto_price_tag',
#                           'auto_price_type','auto_title_all','item_post_date','item_renew_date','is_new','item_show_num','photo','photoList','rank'

df2=pd.DataFrame(columns=['id','car_mileage','car_year','car_price','car_cylinderVolumn','car_color','car_gearType','car_dirveMode','car_fuel','car_door','car_seat'])

r1=requests.get(url_in,headers=headers)

soup1=BeautifulSoup(r1.text,'lxml')

carDetail=soup1.find('div',class_='buyCarDetailTitleTextContent')

carPrice=soup1.find('div',class_='buyCarDetailTitleTextPrice')

car_price=carPrice.find('b').text

info=carDetail.find_all('span')


car_year=info[3].text
car_cylinderVolumn=info[4].text
car_color=info[5].text
car_gearType=info[6].text
car_driveMode=info[7].text
car_fuel=info[8].text
car_mileage=info[9].text
car_door=info[10].text
car_seat=info[11].text
df2= df2.append({'id':573699,'car_mileage':car_mileage,'car_year':car_year,'car_price':car_price,
                 'car_cylinderVolumn':car_cylinderVolumn,'car_color':car_color,'car_gearType':car_gearType,
                 'car_dirveMode':car_driveMode,'car_fuel':car_fuel,'car_door':car_door,'car_seat':car_seat}, ignore_index=True)
df2



,id,car_mileage,car_year,car_price,car_cylinderVolumn,car_color,car_gearType,car_dirveMode,car_fuel,car_door,car_seat
0,573699,140000,2012,31.8萬,1800,白色系,手自排,2WD,汽油車,4門,5
